In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key='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',
    model_name="llama-3.1-70b-versatile",
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [2]:
%pip install -qU langchain_community beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-40929")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.




Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsFIND A STOREBECOME A MEMBERSIGN UP FOR EMAILSEND US FEEDBACKSTUDENT DISCOUNTSGET HELPGET HELPOrder StatusDeliveryReturnsPayment OptionsC

In [29]:
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of website.
    Your job is to extract the job postings and return them in JSON format containing the
    following keys: 'role', 'experience', 'skills', and 'description'.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)
type(res.content)

Unfortunately, the provided text does not contain any job postings. It appears to be a list of Nike's departments, teams, and locations, but does not include any specific job openings.

However, I can provide you with a template of what the JSON output would look like if there were job postings:

```json
[
  {
    "role": "",
    "experience": "",
    "skills": "",
    "description": ""
  }
]
```

If you provide the actual job postings, I can help you extract the relevant information and format it into JSON.


str

In [41]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': '', 'experience': '', 'skills': '', 'description': ''}]

In [42]:
type(json_res)

list

In [9]:
pip install pandas

   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
    --------------------------------------- 0.3/11.5 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.5 MB 1.3 MB/s eta 0:00:09
   -- ------------------------------------- 0.8/11.5 MB 1.3 MB/s eta 0:00:09
   --- ------------------------------------ 1.0/11.5 MB 1.3 MB/s eta 0:00:08
   ---- ----------------------------------- 1.3/11.5 MB 1.3 MB/s eta 0:00:08
   ----- ---------------------------------- 1.6/11.5 MB 1.3 MB/s eta 0:00:08
   ------ --------------------------------- 1.8/11.5 MB 1.3 MB/s eta 0:00:08
   ------- -------------------------------- 2.1/11.5 MB 1.3 MB/s eta 0:00:08
   -------- ------------------------------- 2.4/11.5 MB 1.3 MB/s eta 0:00:08
   --------- ------------------------------ 2.6/11.5 MB 1.3 MB/s eta 0:00:07
   ---------- ----------------------------- 3.1/11.5 MB 1.3 MB/s eta 0:00:07
   ----------- ---------------------------- 3.4/11.5 MB 1.3 MB/s eta 0:00:07
   ----------

In [10]:
import pandas as pd
df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [11]:
import chromadb
import uuid

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                      metadatas={"links": row["Links"]},
                      ids=[str(uuid.uuid4())])

In [12]:
links = collection.query(query_texts=["Experience in Python", "Expertise in React"], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [48]:
job = json_res
job['skills']

TypeError: list indices must be integers or slices, not str

In [14]:
job

[{'role': '', 'experience': '', 'skills': [], 'description': ''}]

In [49]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

TypeError: list indices must be integers or slices, not str

In [12]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Ayush, a software development engineer at McKinsey & Company. McKinsey & Company is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of McKinsey & Company 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase McKinsey & Company portfolio: {link_list}
        Remember you are Ayush, SDE at McKinsey & Company. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Unlock Data-Driven Insights with McKinsey & Company for Converse's Web Analytics Needs

Dear Hiring Manager,

I came across the job posting for Lead, Global Web Analytics at Converse, and I'm excited to introduce McKinsey & Company as a potential partner to support your web analytics needs. As a software development engineer at McKinsey & Company, I'd like to highlight our capabilities in delivering data-driven solutions that can help Converse improve web analytics insights, marketing performance, and consumer behavior understanding.

Our expertise in web analytics, digital marketing, and site optimization aligns perfectly with the requirements of the role. We have extensive experience in leveraging tools like Adobe Analytics, Google Analytics, and tag management tools (e.g., GTM, DTM) to drive recommendations and improve marketing performance. Our team is also well-versed in digital media, performance marketing, and emerging channels such as paid search, paid social, email, a